# Database 2

In [1]:
import sqlite3
import pandas as pd

In [2]:
# ignore this cell (it's just to make certain text red later, but you don't need to understand it).
from IPython.core.display import HTML
HTML('<style>em { color: red; }</style>')

In [3]:
c = sqlite3.connect("movies.db")
c

In [4]:
pd.read_sql("select * from sqlite_master", c)

,type,name,tbl_name,rootpage,sql
0,table,movies,movies,2,"CREATE TABLE ""movies"" (\n""Title"" TEXT,\n ""Dir..."


In [9]:
pd.read_sql("select * from movies", c).head(10)

,Title,Director,Year,Runtime,Rating,Revenue
0,Guardians of the Galaxy,James Gunn,2014,121,8.1,333.13
1,Prometheus,Ridley Scott,2012,124,7.0,126.46
2,Split,M. Night Shyamalan,2016,117,7.3,138.12
3,Sing,Christophe Lourdelet,2016,108,7.2,270.32
4,Suicide Squad,David Ayer,2016,123,6.2,325.02
5,The Great Wall,Yimou Zhang,2016,103,6.1,45.13
6,La La Land,Damien Chazelle,2016,128,8.3,151.06
7,Mindhorn,Sean Foley,2016,89,6.4,0.00
8,The Lost City of Z,James Gray,2016,141,7.1,8.01
9,Passengers,Morten Tyldum,2016,116,7.0,100.01


In [11]:
def qry(sql, cap=10):
    return pd.read_sql(sql, c).head(cap)

In [15]:
qry("""
SELECT *
FROM movies
""", 3)

,Title,Director,Year,Runtime,Rating,Revenue
0,Guardians of the Galaxy,James Gunn,2014,121,8.1,333.13
1,Prometheus,Ridley Scott,2012,124,7.0,126.46
2,Split,M. Night Shyamalan,2016,117,7.3,138.12


# Review: Simple Selections

### Which *movie* has the *highest rating*?

In [18]:
qry("""
SELECT Title
FROM movies
ORDER BY Rating DESC
LIMIT 1
""")

,Title
0,The Dark Knight


### Which *director* made the *shortest movie*?

In [21]:
qry("""
SELECT director
FROM movies
ORDER BY Runtime
LIMIT 1
""")

,Director
0,Claude Barras


### Which *director* made the *highest-revenue movie*?

In [24]:
qry("""
SELECT director
FROM movies
ORDER BY revenue DESC
LIMIT 1
""")

,Director
0,J.J. Abrams


### Which *movie* had the *highest revenue* in *2016*?

In [25]:
qry("""
SELECT title
FROM movies
WHERE year = 2016
ORDER BY revenue DESC
LIMIT 1
""")

,Title
0,Rogue One


### Which *3 movies* had the *highest revenues* in *2016*?

In [26]:
qry("""
SELECT title
FROM movies
WHERE year = 2016
ORDER BY revenue DESC
LIMIT 3
""")

,Title
0,Rogue One
1,Finding Dory
2,Captain America: Civil War


### Which *3 movies* have the *highest rating-to-revenue ratios*?

Introduce `AS`

In [30]:
qry("""
SELECT *, Rating/Revenue AS ratio
FROM movies
ORDER BY ratio DESC
LIMIT 3
""")

,Title,Director,Year,Runtime,Rating,Revenue,ratio
0,Wakefield,Robin Swicord,2016,106,7.5,0.01,750.0
1,"Love, Rosie",Christian Ditter,2014,102,7.2,0.01,720.0
2,Lovesong,So Yong Kim,2016,84,6.4,0.01,640.0


# Aggregate Queries

```
SUM, AVG, COUNT, MIN, MAX
```

### How many *movies* are there?

In [34]:
qry("""
SELECT COUNT(title)
FROM movies
""")

,COUNT(*)
0,998


In [39]:
qry("""
select count(*)
from MOVIES
""")

,count(*)
0,998


### How many *directors* are there?

In [33]:
qry("""
SELECT COUNT(DISTINCT director)
FROM movies
""")

,COUNT(DISTINCT director)
0,643


### What is the *total revenue* of *all the movies*?

In [40]:
qry("""
SELECT SUM(Revenue)
FROM movies
""")

,SUM(Revenue)
0,72215.45


### What is the *average revenue* across *all movies*?

* v1: with `SUM` and `COUNT`
* v2: with `AVG`

In [45]:
qry("""
SELECT SUM(Revenue) / COUNT(*)
FROM movies
""")

,SUM(Revenue) / COUNT(*)
0,72.36017


In [42]:
qry("""
SELECT AVG(Revenue)
FROM movies
""")

,AVG(Revenue)
0,72.36017


### What is the *average revenue* and *average runtime* of *all the movies*?

In [46]:
qry("""
SELECT AVG(Revenue), AVG(Runtime)
FROM movies
""")

,AVG(Revenue),AVG(Runtime)
0,72.36017,113.170341


### What is the *average runtime* for a *James Gunn* movie?

In [47]:
qry("""
SELECT AVG(Runtime)
FROM movies
WHERE Director = "James Gunn"
""")

,AVG(Runtime)
0,104.0


### What is the *average revenue* for a *Ridley Scott* movie?

### *How many movies* were there in *2016*?

In [48]:
qry("""
SELECT COUNT(*)
FROM movies
WHERE Year = 2016
""")

,COUNT(*)
0,296


### What *percentage* of the *total revenue* came from the *highest-revenue movie*?

In [50]:
qry("""
SELECT MAX(Revenue) / SUM(Revenue) * 100
FROM movies
""")

,MAX(Revenue) / SUM(Revenue) * 100
0,1.296994


### What *percentage* of the *total revenue* came from the *highest-revenue movie* in *2016*?

In [51]:
qry("""
SELECT MAX(Revenue) / SUM(Revenue) * 100
FROM movies
WHERE Year = 2016
""")

,MAX(Revenue) / SUM(Revenue) * 100
0,4.746581


# GROUP BY Queries

```sql
SELECT ???, ??? FROM Movies
GROUP BY ???
```

### What is the *total revenue* per each *year*?

* v1: the amounts
* v2: the amounts, as labeled by year

In [52]:
qry("""
SELECT SUM(Revenue)
FROM movies
GROUP BY year
""")

,SUM(Revenue)
0,3624.46
1,4306.23
2,5053.22
3,5292.26
4,5989.65
5,5431.96
6,6910.29
7,7544.21
8,7997.40
9,8854.12


In [53]:
# better version:
qry("""
SELECT year, SUM(Revenue)
FROM movies
GROUP BY year
""")

,Year,SUM(Revenue)
0,2006,3624.46
1,2007,4306.23
2,2008,5053.22
3,2009,5292.26
4,2010,5989.65
5,2011,5431.96
6,2012,6910.29
7,2013,7544.21
8,2014,7997.40
9,2015,8854.12


### *How many movies* were by each *director*?

In [57]:
qry("""
SELECT director, COUNT(*) as count
FROM movies
GROUP BY director
ORDER BY count DESC
""", 15)

,Director,count
0,Ridley Scott,8
1,David Yates,6
2,M. Night Shyamalan,6
3,Michael Bay,6
4,Paul W.S. Anderson,6
5,Antoine Fuqua,5
6,Christopher Nolan,5
7,Danny Boyle,5
8,David Fincher,5
9,Denis Villeneuve,5


### What is the *average rating* for each *director*?

### What is the *average runtime* for each *director*?

### How many *unique directors* created a movie in each *year*

In [58]:
qry("""
SELECT year, COUNT(DISTINCT director)
FROM movies
GROUP BY year
""")

,Year,COUNT(DISTINCT director)
0,2006,44
1,2007,51
2,2008,51
3,2009,51
4,2010,60
5,2011,63
6,2012,64
7,2013,88
8,2014,97
9,2015,127


# Combining GROUP BY with other CLAUSES

<img src="groupby.png">

### What is the *total revenue* of per *year*, in *recent* years?

### Which *directors* have had the *largest number of movies* earning *over 100M dollars*?

### Which *three* of the *directors* have the *greatest average rating*?

In [61]:
qry("""
SELECT director, AVG(rating) AS avg_rating, COUNT(*) AS mov_count
FROM movies
GROUP BY director
ORDER BY avg_rating DESC
LIMIT 3
""")

,Director,avg_rating,mov_count
0,Nitesh Tiwari,8.80,1
1,Christopher Nolan,8.68,5
2,Makoto Shinkai,8.60,1


Why is the above question maybe not the best to ask?

### Which *three* of the *directors* have the *greatest average rating* over at *least three movies*?

In [63]:
qry("""
SELECT director, AVG(rating) AS avg_rating, COUNT(*) AS mov_count
FROM movies
GROUP BY director
HAVING mov_count >= 3
ORDER BY avg_rating DESC
LIMIT 3
""")

,Director,avg_rating,mov_count
0,Christopher Nolan,8.68,5
1,Martin Scorsese,7.92,5
2,Quentin Tarantino,7.90,4


Need filtering BEFORE and AFTER the GROUP operations

<img src="pipeline.png">

# WHERE vs. HAVING

* WHERE: filter rows in original table
* HAVING: filter groups

### Repeat: Which *three* of the *directors* have the *greatest average rating* over at *least three movies*?

<img src="having.png">

### Which *directors* have had *more than 5 movies* that have been *since 2014*?

In [73]:
qry("""
SELECT director, COUNT(*) as mov_count
FROM movies
WHERE year > 2013
GROUP BY director
HAVING mov_count >= 3
ORDER BY mov_count DESC
""")

,Director,mov_count
0,Antoine Fuqua,3
1,Mike Flanagan,3
2,Nicholas Stoller,3


### Which *directors* have had more than *five* movies with runtimes under *100* minutes

In [ ]:
c.close()